In [1]:
import findspark
findspark.init()
findspark.find()

import pyspark
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext("local", "My Application")

In [3]:
raw_rdd = sc.textFile("car_data.csv") #THE FIRST LINE IS HEADERS!!!!!!!!
raw_rdd.take(5)

['1,I,VXIO456XLBB630221,Nissan,Altima,2003,2002-05-08,Initial sales from TechMotors',
 '2,I,INU45KIOOPA343980,Mercedes,C300,2015,2014-01-01,Sold from EuroMotors',
 '3,A,VXIO456XLBB630221,,,,2014-07-02,Head on collision',
 '4,R,VXIO456XLBB630221,,,,2014-08-05,Repair transmission',
 '5,I,VOME254OOXW344325,Mercedes,E350,2015,2014-02-01,Sold from Carmax']

In [4]:
def extract_vin_key_value(x):
    x = list(x.split(","))
    key_value = (x[2], (x[3], x[4], x[5]))
#     key_value = (x[2], (x[3]))

    return key_value

In [5]:
# Iterates through each line and applies this function
# extract the key and the value
vin_kv = raw_rdd.map(lambda x: extract_vin_key_value(x))
vin_kv.take(50)

[('VXIO456XLBB630221', ('Nissan', 'Altima', '2003')),
 ('INU45KIOOPA343980', ('Mercedes', 'C300', '2015')),
 ('VXIO456XLBB630221', ('', '', '')),
 ('VXIO456XLBB630221', ('', '', '')),
 ('VOME254OOXW344325', ('Mercedes', 'E350', '2015')),
 ('VOME254OOXW344325', ('', '', '')),
 ('VXIO456XLBB630221', ('', '', '')),
 ('EXOA00341AB123456', ('Mercedes', 'SL550', '2016')),
 ('VOME254OOXW344325', ('', '', '')),
 ('VOME254OOXW344325', ('', '', '')),
 ('EXOA00341AB123456', ('', '', '')),
 ('EXOA00341AB123456', ('', '', '')),
 ('VOME254OOXW344325', ('', '', '')),
 ('UXIA769ABCC447906', ('Toyota', 'Camery', '2017')),
 ('UXIA769ABCC447906', ('', '', '')),
 ('INU45KIOOPA343980', ('', '', ''))]

In [6]:
def populate_make(list_):
    print("check")
    buffer = []
    out_list = []
    groupMake = None
    groupYear = None
    for record in list_:
        print(record)
        type, make, year = record.split(',')
        buffer.append((type, make, year))
        if groupMake == None:
            groupMake = make
        if groupYear == None:
            groupYear = year
    for item in buffer:
        #out_list.append()
        if (item[0] == 'A'):
            out_list.append("{},{}".format(groupMake, groupYear))
    return out_list

In [8]:
enhance_make = vin_kv.groupByKey().flatMap(lambda kv: populate_make(kv[1][0]))
# enhance_make = vin_kv.groupByKey().collect()
# enhance_make.collect()


# enhance_make = vin_kv.groupByKey().flatMap(lambda kv: list(kv[1]))
# enhance_make = vin_kv.groupByKey().map(lambda kv: list(kv[1][0])).collect()
# enhance_make.take(50)
# enhance_make.collect()
# vin_kv.groupByKey().map(lambda kv: list(kv[1])).collect()


In [26]:
def extract_make_key_value(x):
    x = list(x.split(","))
    key_value = ((x[1][0], x[1][2]), 1)

    return key_value

In [10]:
# Kartik
def extract_make_key_value(line):
    fields = line.strip().split(",")
    make = fields[0]
    year = fields[1]
    key = "{}-{}".format(make, year)
    value = 1
    return (key, value)

In [11]:
make_kv = enhance_make.map(lambda x: extract_make_key_value(x))

In [21]:
final = make_kv.reduceByKey(lambda x:(x[0],x[1]))

In [19]:
# for t in final.collect():
#     print(t[0], [v for v in t[1]])

In [ ]:
# https://backtobazics.com/big-data/spark/apache-spark-groupbykey-example/